In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'d': 0, 'h': 1, 'r': 2, 'e': 3, 'l': 4, ' ': 5, 'o': 6, 'w': 7},
 {0: 'd', 1: 'h', 2: 'r', 3: 'e', 4: 'l', 5: ' ', 6: 'o', 7: 'w'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([1, 3, 4, 4, 6, 5, 7, 6, 2, 4], [3, 4, 4, 6, 5, 7, 6, 2, 4, 0])

In [9]:
sample("h", 4)

'h ll'

In [13]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    hs[-1] = h
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t - 1] * dh_dz1.T * dL_dh.T
        dL_dbh += dh_dz1 * dL_dh

        # raise RuntimeError

        # dF_dh = dL_dh * (dh_dz1 @ Whh)
        dF_dh = dL_dh @ (dh_dz1 * Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)
        # np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 0.040794031434180575
[Loss]: 0.03876195503839716
[Loss]: 0.03820365797799319
[Loss]: 0.03999122058794655
[Loss]: 0.042103829528977066
[Loss]: 0.04522776214553356
[Loss]: 0.055235411341373136
[Loss]: 0.11253054461134837
[Loss]: 4.059742350521636
[Loss]: 0.03725316581204526
[Loss]: 25.129761149606594
[Loss]: 6.145578231257514
[Loss]: 22.812728276628068
[Loss]: 1.5506217356399319
[Loss]: 0.036959840591187
[Loss]: 0.041052354981221756
[Loss]: 0.02985188191916791
[Loss]: 0.03717542054363595
[Loss]: 0.030249471324755358
[Loss]: 0.05341113373646717
[Loss]: 0.02251650446542426
[Loss]: 0.02150103486446526
[Loss]: 0.020640864997418393
[Loss]: 0.02039432361491233
[Loss]: 6.278200427048614
[Loss]: 0.022554688010496465
[Loss]: 0.022409720394142775
[Loss]: 0.02274680990301151
[Loss]: 0.023282156530493592
[Loss]: 0.02412240137093321
[Loss]: 0.02499212620267554
[Loss]: 0.025876337625106424
[Loss]: 0.026814028969797257
[Loss]: 0.03398760347345845
[Loss]: 0.04407653064061701
[Loss]: 0.0419604724

In [14]:
sample("h", 10)

'ello world'